In [226]:
# !pip3.10 install -r requirements.txt
# !pip3.10 install transformers -U

In [227]:
from transformers import pipeline
import pandas as pd

######################
# TinyBERT
######################
# Fastest
# Pretrained on MNLI
# Takes about 0.4seconds to initialize and about <0.1s to classify
# Untrained model, requires fine-tuning
# 14.5 million parameters
# https://huggingface.co/huawei-noah/TinyBERT_General_4L_312D
# classifier = pipeline("zero-shot-classification",
#                       model="huawei-noah/TinyBERT_General_4L_312D")

######################
# bart-large-mnli
######################
# Slower but robust, works out of the box
# Pretrained on MNLI
# Takes about 50seconds to initialize and about 4s to classify
# Made for NLI (natural language inference) classification
# 420 million parameters
# https://huggingface.co/facebook/bart-large-mnli
# classifier = pipeline("zero-shot-classification",
#                       model="facebook/bart-large-mnli")

######################
# nli-roberta-base
######################
# Takes about 4 to initialize and about 0.1s to classify, works out of the box
# Pretrained on MNLI
# Made for NLI (natural language inference) classification
# 120 million parameters
# https://huggingface.co/cross-encoder/nli-roberta-base
classifier = pipeline("zero-shot-classification",
                      model="cross-encoder/nli-roberta-base")

# Not meant for classification, final layer is not trained and intialized with random weights
# classifier = pipeline("zero-shot-classification",
#                       model="meta-llama/Llama-3.2-1B")

Device set to use mps:0


In [228]:
train_data = pd.read_csv("/Users/lekshmanannatarajan/Library/CloudStorage/GoogleDrive-zuari1993@gmail.com/My Drive/Colab Notebooks/LLM_BART_classifier/data_local/outputs/synthetic_test_prompts.csv")
train_data

,prompt,truth_label
0,I'm stuck on this project and need some fresh ...,help me brainstorm
1,I'm drawing a blank. Can we brainstorm some po...,help me brainstorm
2,I need help brainstorming ideas for a research...,help me brainstorm
3,Can you help me brainstorm some creative marke...,help me brainstorm
4,I'm feeling uninspired. Help me brainstorm som...,help me brainstorm
...,...,...
74,Can you summarize this report on the latest tr...,summarize this report for me
75,I don't understand this complex mathematical p...,summarize this report for me
76,Summarize this report on the impact of video g...,summarize this report for me
77,I need a summary of this report on the benefit...,summarize this report for me


In [229]:
candidate_labels = list(train_data.truth_label.unique())
classified_pred = []
classified_pred_prob = []

for prompt in train_data.prompt.to_list():
    prompt_to_classify =   prompt

    results = classifier(prompt_to_classify, candidate_labels)
    max_score_index = results['scores'].index(max(results['scores']))
    highest_score_label = results['labels'][max_score_index]
    
    classified_pred.append(highest_score_label)
    classified_pred_prob.append(max(results['scores']))
    

In [230]:
train_data['classified_pred'] = classified_pred
train_data['classified_data_prob'] = classified_pred_prob

train_data

,prompt,truth_label,classified_pred,classified_data_prob
0,I'm stuck on this project and need some fresh ...,help me brainstorm,help me brainstorm,0.898165
1,I'm drawing a blank. Can we brainstorm some po...,help me brainstorm,help me brainstorm,0.886622
2,I need help brainstorming ideas for a research...,help me brainstorm,help me brainstorm,0.928938
3,Can you help me brainstorm some creative marke...,help me brainstorm,help me brainstorm,0.942887
4,I'm feeling uninspired. Help me brainstorm som...,help me brainstorm,help me brainstorm,0.977576
...,...,...,...,...
74,Can you summarize this report on the latest tr...,summarize this report for me,help me brainstorm,0.769999
75,I don't understand this complex mathematical p...,summarize this report for me,summarize this report for me,0.610896
76,Summarize this report on the impact of video g...,summarize this report for me,summarize this report for me,0.669737
77,I need a summary of this report on the benefit...,summarize this report for me,summarize this report for me,0.635843


In [231]:
from sklearn.metrics import accuracy_score

accuracy_score_ = accuracy_score(train_data.truth_label, train_data.classified_pred, normalize=True)

print("Model accuracy in classification = ", accuracy_score_*100, "%")

Model accuracy in classification =  88.60759493670885 %
